In [ ]:
"""
Created on Fri Jun 4 15:50 2020

This is a script to convert the NEMO temperature and salinity to potential temperature and practical salinity

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gsw
import matplotlib.pyplot as plt
import multimelt.useful_functions as uf

import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=18, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'bf663' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_mask = xr.open_dataset(inputpath_data+'mask_variables_of_interest_allyy_Ant_stereo.nc')#, chunks={'x': 600, 'y': 600})
file_mask2 = xr.open_dataset(inputpath_data+'mask_depth_coord_Ant_stereo.nc')


In [ ]:
file_isf_1970 = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1970.nc')
file_isf_1970_cutted = uf.cut_domain_stereo(file_isf_1970, map_lim, map_lim)#.squeeze().drop('time')

file_mask_cutted = uf.cut_domain_stereo(file_mask, map_lim, map_lim)#.squeeze().drop('time')


In [ ]:
lon = file_isf_1970_cutted['longitude']
lat = file_isf_1970_cutted['latitude']

Prepare the depth axis

In [ ]:
nemo_depth = np.round(file_mask2['gdept_0'].squeeze(dim=['lon','lat']), 3) # round to mm scale - should be enough

Cut out the temperature and salinity and assign the new depth axis

CONVERT ABSOLUTE SALIINITY TO PRACTICAL SALINITY IN OPEN OCEAN REGIONS

In [ ]:
for tt,timet in enumerate(range(1970,2051)): 
    print(timet)
    
    ds_ts  = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo_'+str(timet)+'.nc') #, chunks={'x': 600, 'y': 600})
    ds_ts_cutted = uf.cut_domain_stereo(ds_ts, map_lim, map_lim)
    ds_temp_saline_input = ds_ts_cutted[['thetao', 'so']]
    ds_temp_saline_input = ds_temp_saline_input.rename({'thetao': 'temperature', 'so': 'salinity'})
    ds_temp_saline_input = ds_temp_saline_input.rename({'deptht': 'depth'})
    ds_temp_saline_input['depth'] = np.round(ds_temp_saline_input.depth, 3)
    ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)
    
    file_isf = xr.open_dataset(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_'+str(timet)+'.nc')
    file_isf_cutted = uf.cut_domain_stereo(file_isf, map_lim, map_lim)#.squeeze().drop('time')
    mask_ocean = file_isf_cutted['ISF_mask'] == 1  #ocean without ice shelf cavity
    
    ds_temp_saline_input['theta_ocean'] = ds_temp_saline_input['temperature'].where(mask_ocean) 
    ds_temp_saline_input['salinity_ocean'] = xr.apply_ufunc(gsw.SP_from_SA, ds_temp_saline_input['salinity'].where(mask_ocean), ds_temp_saline_input['depth'], lon, lat, dask = 'allowed')
    ds_temp_saline_output = ds_temp_saline_input[['theta_ocean', 'salinity_ocean']]
    ds_temp_saline_output.to_netcdf(inputpath_profiles + 'T_S_theta_ocean_corrected_'+str(timet)+'.nc')

NOW CONTINUE WITH T_S_profile_front_Smith.ipynb